# Jack's Car Rental: Value Iteration
Example 4.2 from *Reinforcement Learning: An Introduction* by Sutton and Barto.

This example from Sutton and Barto's excellent book is trickier to implement than you might think. I've built 2 versions: one using policy iteration, and the other with value iteration.

For policy iteration, note how policy evaluation becomes shorter and shorter as the policy converges. It is also interesting to note that value iteration is slower than policy iteration in this example. Value iteration must check all possible actions from every state in each iteration.

Both methods converge to the result shown in the textbook.

Python Notebook by Patrick Coady: [Learning Artificial Intelligence](https://learningai.io/)

In [1]:
import numpy as np
import math
import time

In [2]:
def clipped_poisson(lam, max_k):
    """
    Return poisson PMF clipped at max_k with remaining tail probability
    placed at max_k.
    """
    pmf = np.zeros(max_k + 1)
    for k in range(max_k):
        pmf[k] = math.exp(-lam) * lam**k / math.factorial(k)
    pmf[max_k] = 1 - np.sum(pmf)
    
    return pmf       

In [3]:
def build_rent_return_pmf(lambda_request, lambda_return, max_cars):
    """
    Return p(new_rentals, returns | initial_cars) as numpy array:
        p[initial_cars, new_rentals, returns]
    """
    pmf = np.zeros((max_cars+1, max_cars+1, max_cars+1))                                # why +1?
    
    for init_cars in range(max_cars + 1):
        new_rentals_pmf = clipped_poisson(lambda_request, init_cars)
        for new_rentals in range(init_cars + 1):
            max_returns = max_cars - init_cars + new_rentals
            returns_pmf = clipped_poisson(lambda_return, max_returns)
            for returns in range(max_returns + 1):
                p = returns_pmf[returns] * new_rentals_pmf[new_rentals]
                pmf[init_cars, new_rentals, returns] = p
                
    return pmf

In [4]:
class JacksWorld(object):
    """Environment model of Jack's Car Rental"""
    def __init__(self, lambda_return1, lambda_return2,
                 lambda_request1, lambda_request2, max_cars):
        # pre-build the rentals/returns pmf for each location
        self.rent_return_pmf = []
        self.rent_return_pmf.append(build_rent_return_pmf(lambda_request1,
                                                      lambda_return1,
                                                      max_cars))
        self.rent_return_pmf.append(build_rent_return_pmf(lambda_request2,
                                                      lambda_return2,
                                                      max_cars))
        self.max_cars = max_cars
        
    def get_transition_model(self, s, a):
        """
        Return 2-tuple:
            1. p(s'| s, a) as dictionary:
                keys = s'
                values = p(s' | s, a)
            2. E(r | s, a, s') as dictionary:
                keys = s'
                values = E(r | s, a, s')
        """
        s = (s[0] - a, s[1] + a)         # move a cars from loc1 to loc2
        move_reward = -math.fabs(a) * 2  # ($2) per car moved
        t_prob, expected_r = ([{}, {}], [{}, {}])
        for loc in range(2):
            morning_cars = s[loc]
            rent_return_pmf = self.rent_return_pmf[loc]
            for rents in range(morning_cars + 1):
                max_returns = self.max_cars - morning_cars + rents
                for returns in range(max_returns + 1):
                    p = rent_return_pmf[morning_cars, rents, returns]
                    if p < 1e-5:
                        continue
                    s_prime = morning_cars - rents + returns
                    r = rents * 10
                    t_prob[loc][s_prime] = t_prob[loc].get(s_prime, 0) + p
                    expected_r[loc][s_prime] = expected_r[loc].get(s_prime, 0) + p * r
        
        # join probabilities and expectations from loc1 and loc2
        t_model, r_model = ({}, {})
        for s_prime1 in t_prob[0]:
            for s_prime2 in t_prob[1]:
                p1 = t_prob[0][s_prime1]  # p(s' | s, a) for loc1
                p2 = t_prob[1][s_prime2]  # p(s' | s, a) for loc2
                t_model[(s_prime1, s_prime2)] = p1 * p2
                # expectation of reward calculated using p(s', r | s, a)
                # need to normalize by p(s' | s, a)
                norm_E1 = expected_r[0][s_prime1] / p1
                norm_E2 = expected_r[1][s_prime2] / p2
                r_model[(s_prime1, s_prime2)] = norm_E1 + norm_E2 + move_reward
                
        return t_model, r_model

In [5]:
# Initialize environment

max_cars = 20
jacks = JacksWorld(lambda_return1=3, lambda_return2=2,
                 lambda_request1=3, lambda_request2=4, max_cars=max_cars)

In [6]:
# Initialize value function

V = np.zeros((max_cars+1, max_cars+1))
states = [(s0, s1) for s0 in range(max_cars+1) for s1 in range(max_cars+1)]
gamma = 0.9

start_time = time.time()
# Value Iteration
theta = 0.5          # V(s) delta stopping threshold
print('Worst |V_old(s) - V(s)| delta:')
for k in range(100):
    delta = 0
    V_old = V.copy()
    V = np.zeros((max_cars+1, max_cars+1))
    for s in states:
        v_best = -1000
        max_a = min(5, s[0], max_cars-s[1])
        min_a = max(-5, -s[1], -(max_cars-s[0]))
        for a in range(min_a, max_a+1):
            t_model, r_model = jacks.get_transition_model(s, a)            
            v_new = 0
            for s_prime in t_model:
                p = t_model[s_prime]
                r = r_model[s_prime]
                # must use previous iteration's V(s): V_old(s)
                v_new += p * (gamma * V_old[s_prime] + r)
            v_best = max(v_best, v_new)
        V[s] = v_best
        delta = max(delta, abs(V[s] - V_old[s]))
    print('Iteration {}: max delta = {:.2f}'.format(k, delta))
    if delta < theta: break

# Extract Policy from V(s)
pi = np.zeros((max_cars+1, max_cars+1), dtype=np.int16)
for s in states:
    best_v = -1000
    max_a = min(5, s[0], max_cars-s[1])
    min_a = max(-5, -s[1], -(max_cars-s[0]))
    for a in range(min_a, max_a+1):
        t_model, r_model = jacks.get_transition_model(s, a)
        v = 0
        for s_prime in t_model:
            p = t_model[s_prime]
            r = r_model[s_prime]
            v += p * (gamma * V[s_prime] + r)
        if v > best_v:
            pi[s] = a
            best_v = v
            
print('\nValue iteration done, final policy:')            
print(pi)
            
print("\n--- {:.2f} seconds ---".format(time.time() - start_time))

Worst |V_old(s) - V(s)| delta:
Iteration 0: max delta = 69.98
Iteration 1: max delta = 62.97
Iteration 2: max delta = 56.64
Iteration 3: max delta = 50.83
Iteration 4: max delta = 45.39
Iteration 5: max delta = 40.37
Iteration 6: max delta = 35.82
Iteration 7: max delta = 31.71
Iteration 8: max delta = 28.00
Iteration 9: max delta = 24.65
Iteration 10: max delta = 21.65
Iteration 11: max delta = 18.99
Iteration 12: max delta = 16.64
Iteration 13: max delta = 14.61
Iteration 14: max delta = 12.84
Iteration 15: max delta = 11.31
Iteration 16: max delta = 9.99
Iteration 17: max delta = 8.84
Iteration 18: max delta = 7.84
Iteration 19: max delta = 6.97
Iteration 20: max delta = 6.20
Iteration 21: max delta = 5.53
Iteration 22: max delta = 4.94
Iteration 23: max delta = 4.42
Iteration 24: max delta = 3.96
Iteration 25: max delta = 3.55
Iteration 26: max delta = 3.18
Iteration 27: max delta = 2.86
Iteration 28: max delta = 2.56
Iteration 29: max delta = 2.30
Iteration 30: max delta = 2.07
It

### Digging Around

In [10]:
lambda_return1=3
lambda_return2=2
lambda_request1=3 
lambda_request2=4
max_cars=20

rent_return_pmf = []
rent_return_pmf.append(build_rent_return_pmf(lambda_request1,
                                                      lambda_return1,
                                                      max_cars))
rent_return_pmf.append(build_rent_return_pmf(lambda_request2,
                                                      lambda_return2,
                                                      max_cars))

In [15]:
rent_return_pmf[0].shape, rent_return_pmf[1].shape

((21, 21, 21), (21, 21, 21))

In [10]:
rent_return_pmf

[array([[[4.97870684e-02, 1.49361205e-01, 2.24041808e-01, ...,
          3.01271548e-09, 4.75691918e-10, 8.31441582e-11],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         ...,
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],
 
        [[2.47875218e-03, 7.43625653e-03, 1.11543848e-02, ...,
          1.49994272e-10, 2.78228105e-11, 0.00000000e+00],
         [4.73083162e-02, 1.41924949e-01, 2.12887423e-01, ...,
          2.86272121e-09, 4.52008612e-10, 7.90046543e-11],
       